In [ ]:
# This uses an earlier audio clip

# FILE_MUS = 'test0.mp3'
# FILE_CSV = 'test0.csv'

# # adjust until good alignment
# OFFSET = 848
# STRETCH = -.00003
# LEFT_TEST = 420, 620
# RIGHT_TEST = 3850, 3950

# TRIM_START = 400

# SCATTER = [
#     [580, 1300, 83, 'r'],
#     [1550, 2200, 82, 'g'],
#     [2200, 2800, 80, 'b'],
#     [2800, 3450, 78, 'k'],
# ]

In [ ]:
FILE_MUS = 'test1.mp3'
FILE_CSV = 'test1.csv'

# adjust until good alignment
OFFSET = -490
STRETCH = -.00004
LEFT_TEST = 50, 190
RIGHT_TEST = 9300, 9600

TRIM_START = 30

SCATTER = [
    [200, 1675, 72, '#9f1d3f', [368, 558, 726, 927, 1117, 1307, 1508], 'C'],
    [1675, 2994, 74, '#eb6437', [1832, 2002, 2172, 2364, 2546, 2693, 2840], 'D'],
    [2994, 4211, 76, '#e3c70e', [3169, 3361, 3497, 3656, 3792, 3962, 4064], 'E'],
    [4211, 5463, 77, '#008a61', [4381, 4540, 4677, 4846, 5016, 5163, 5322], 'F'],
    [6032, 7250, 79, '#77c1fe', [6166, 6323, 6446, 6602, 6758, 6937, 7071], 'G'],
    [7250, 8423, 81, '#0062bf', [7443, 7580, 7714, 7845, 8003, 8137, 8282], 'A'],
    [8423, 9518, 83, '#774fc2', [8888, 9268, 9332], 'B'],
]
LIGHT = {
    '#9f1d3f': '#c46',
    '#eb6437': '#f96',
    '#e3c70e': '#ff3',
    '#008a61': '#3b9',
    '#77c1fe': '#aff',
    '#0062bf': '#39e',
    '#774fc2': '#a7f',
}

In [ ]:
import numpy as np
from numpy import pi
import matplotlib
from matplotlib import pyplot as plt
import librosa
from IPython.display import Audio
import scipy
import csv
# from stats import regression
from sklearn import linear_model

In [ ]:
PAGE_LEN = 2048

HOP_LEN = PAGE_LEN // 4

amp_c, pre_c, freq_c, *_ = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
plt.rcParams.update({
    "text.usetex": not plt.rcParams['text.usetex'],
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    'font.size': 16,
    "legend.framealpha": 1, 
})
print('TEX:', plt.rcParams['text.usetex'])

Run the above ceel to toggle Latex debug mode!

In [ ]:
def sino(freq, length):
    return np.sin(np.arange(length) * freq * TWO_PI / SR)

def play(data):
    return Audio(np.concatenate([data, [1]]), rate = SR)

def findPeaks(energy):
    slope = np.sign(energy[1:] - energy[:-1])
    extrema = slope[1:] - slope[:-1]
    return np.argpartition(
        (extrema == -2) * energy[1:-1], - N_HARMONICS,
    )[- N_HARMONICS:] + 1

def sft(signal, freq_bin):
    # Slow Fourier Transform
    return np.abs(np.sum(signal * np.exp(IMAGINARY_LADDER * freq_bin))) / PAGE_LEN

def refineGuess(guess, signal):
    def loss(x):
        if x < 0:
            return 0
        return - sft(signal, x)
    freq_bin, loss = blindDescend(loss, .01, .4, guess)
    return freq_bin * SR / PAGE_LEN, - loss

def widePlot(h = 3, w = 12):
    plt.gcf().set_size_inches(w, h)

    
def spectro(signal, do_wide = True, trim = 130):
    energy = np.abs(rfft(signal * HANN))
    plt.plot(energy[:trim])
    if do_wide:
        widePlot()

def concatSynth(synth, harmonics, n):
    buffer = []
    for i in range(n):
        synth.eat(harmonics)
        buffer.append(synth.mix())
    return np.concatenate(buffer)

def pitch2freq(pitch):
    return np.exp((pitch + 36.37631656229591) * 0.0577622650466621)

def freq2pitch(f):
    return np.log(f + .001) * 17.312340490667562 - 36.37631656229591

In [ ]:
raw_0, SR = librosa.load(FILE_MUS)
SR

In [ ]:
play(raw_0)

In [ ]:
# help(librosa.yin)
f0s = librosa.yin(raw_0, 200, 2500, SR, PAGE_LEN)
plt.plot(f0s)
widePlot()

In [ ]:
def traceEnergy(signal):
    i = 0
    energy = []
    while True:
        page = signal[i*HOP_LEN : i*HOP_LEN + PAGE_LEN]
        if page.size < PAGE_LEN:
            break
        energy.append(np.sum(scipy.signal.periodogram(page, SR)) / PAGE_LEN)
        i += 1
    return energy
e = np.array(traceEnergy(raw_0))
plt.plot(e)
widePlot()

In [ ]:
ee = (e - 2758.94165039096) * 10000000
plt.plot(ee)
widePlot()

In [ ]:
def getP():
    time = []
    pressure = []
    with open(FILE_CSV, 'r') as f:
        last_t = -1
        epoch = 0
        for t, p in csv.reader(f):
            t = int(t)
            if t < last_t:
                epoch += 1
            last_t = t
            time.append((t + 16384 * epoch) / 1000)
            pressure.append(int(p))
    return time, pressure
t, p = getP()
plt.plot(t, p)
widePlot()

In [ ]:
def sampleP(time, pressure, t):
    s = np.sign(time - t)
    i = np.where(s[1:] - s[:-1])[0][0]
    t4, t5, t6 = time[i], t, time[i+1]
    return pressure[i] * (t6-t5) / (t6-t4) + pressure[i+1] * (t5-t4) / (t6-t4)

def uniformP(time, pressure):
    time = np.array(time)
    t = 0
    result = []
    while True:
#         print(t, end='\r', flush = True)
        t += HOP_LEN / SR + STRETCH
        if t > time[-1]:
            break
        if t < time[0]:
            continue
        result.append(sampleP(time, pressure, t))
#     print('Done                ')
    return np.array(result)
pp = uniformP(t, p)

In [ ]:
if OFFSET > 0:
    eee = ee[OFFSET:]
    ff = f0s[OFFSET:]
    pp_ = pp
else:
    pp_ = pp[-OFFSET:]
    eee = ee
    ff = f0s

In [ ]:
st, en = LEFT_TEST
x = np.arange(en - st) * HOP_LEN / SR
plt.plot(x, eee[st:en] * 3, label='amplitude')
plt.plot(x, pp_[st:en], label='pressure')
widePlot()
plt.xlabel('time (seconds)')
plt.legend()

plt.savefig('imgs/align_left.svg', bbox_inches='tight')

In [ ]:
st, en = RIGHT_TEST
x = np.arange(en - st) * HOP_LEN / SR
plt.plot(x, eee[st:en] * 3, label='amplitude')
plt.plot(x, pp_[st:en], label='pressure')
widePlot()
plt.xlabel('time (seconds)')
plt.legend()

plt.savefig('imgs/align_right.svg', bbox_inches='tight')

In [ ]:
plt.plot(eee[:1500])
widePlot()

In [ ]:
eee = eee[:pp_.size]
ff = ff[:pp_.size]
eeee = eee[TRIM_START:]
fff = ff[TRIM_START:]
ppp = pp_[TRIM_START:]

In [ ]:
ffff = []
for x, y in zip(fff, ppp):
    if y > 15:
        ffff.append(x)
    else:
        ffff.append(0)
ffff = np.array(ffff)
plt.plot(ffff)
widePlot()

In [ ]:
SIZE = eeee.size

In [ ]:
x = np.arange(SIZE) / SR * HOP_LEN
plt.plot(x, eeee * 18, label='amplitude')
plt.plot(x, ppp * 8, label='pressure')
plt.plot(x, ffff, label='frequency')
widePlot(5, 50)
plt.xlabel('time (seconds)')
plt.legend()

# plt.savefig('eyeball.pdf')
eeee.size, ppp.size, ffff.size

In [ ]:
def scatterBend(ax, p, f, start, end, pitch, c):
    p = p[start:end]
    f = f[start:end]
    pb = freq2pitch(f) - pitch - .75
    pp = []
    pbpb = []
    for x, y in zip(p, pb):
        if x > 20:
            pp.append(x)
            pbpb.append(y)
    scatter = ax.scatter(pp, pbpb, c=c, s=.5, marker='.')
    ax.grid(which='major')
    ax.set_ylim([-4,14])
    return scatter


In [ ]:
plt.plot(ffff)
widePlot()


In [ ]:
# octave hysterisis

NOTE = 4
NOTE_I = 1
start, end, pitch, color, mids, symbol = SCATTER[NOTE]
last_start = start
ax = plt.axes()
for i, x in enumerate(mids + [end]):
    if NOTE_I < 0 or i in range(NOTE_I * 2, NOTE_I * 2 + 2):
        if i % 2 == 0:
            sc = scatterBend(ax, ppp, ffff, last_start, x, pitch, 'b')
            sc.set_label('upward')
        else:
            sc = scatterBend(ax, ppp, ffff, last_start, x, pitch, 'r')
            sc.set_label('downward')
    last_start = x
plt.xlabel('pressure (Pa)')
plt.ylabel('pitch bend (semitones)')
lgnd = plt.legend()
for handle in lgnd.legendHandles:
    handle.set_sizes([50])

plt.savefig('imgs/hysteresis.svg', bbox_inches='tight')

In [ ]:
NOTE = 1
NOTE_I = 2
start, end, pitch, color, mids, symbol = SCATTER[NOTE]
last_start = start
for i, x in enumerate(mids + [end]):
    if NOTE_I < 0 or i in range(NOTE_I * 2, NOTE_I * 2 + 2):
        scatterBend(plt.gca(), ppp, ffff, last_start, x, pitch, 'b' if i % 2 == 0 else 'r')
    last_start = x
axes = plt.gca()
axes.set_xlim([0,200])
axes.set_ylim([-2,1.5])
widePlot(4, 10)

## filter (pressure, pitch) pairs with timing. 
So like, invalidate those close to the octave change. 

previously we used unsupervised learning to call two distribution domains. 

legacy code here

In [ ]:
# from sklearn import cluster
# from sklearn import mixture

In [ ]:
# pitch, (X, Y) = regress_data[2]

# # clustering = cluster.DBSCAN(eps=8e4, min_samples=10).fit([*zip(X, Y)])
# # clustering = cluster.SpectralClustering(n_clusters=2).fit([*zip(X, Y)])
# # clustering = cluster.AgglomerativeClustering(n_clusters=2).fit([*zip(X, Y)])
# # clustering = cluster.OPTICS().fit([*zip(X, Y)])
# # clustering = cluster.KMeans(n_clusters=2).fit([*zip(X, Y)])
# # clustering = cluster.MeanShift().fit([*zip(X, Y)])
# # clustering = cluster.Birch(n_clusters=2).fit([*zip(X, Y)])
# # print(clustering.labels_)
# # c = clustering.labels_

# mix = mixture.GaussianMixture(n_components=2, warm_start=False).fit([*zip(X, Y)])
# print('iter', mix.n_iter_, '. if > 100, raise max')
# c = mix.predict([*zip(X, Y)])
# print(mix.means_)

# plt.scatter(X, Y, s=1, c=['brgk'[t] for t in c])
# # plt.scatter(X, Y, s=1, c=['b' if t < 2 else 'r' for t in c])
# # plt.scatter(X, Y, s=1, c=c)

In [ ]:
x = np.arange(SIZE) / SR * HOP_LEN
plt.plot(x[3690:3920], ffff[3690:3920], c=freq_c)
plt.axvspan(86.9, 87.18, facecolor='r', alpha=0.3)
span = plt.axvspan(88.53, 88.9, facecolor='r', alpha=0.3)
plt.xlabel('time (seconds)')
plt.ylabel('frequency (Hz)')
plt.legend([span], ['not in equilibrium'])
# for illustration
plt.savefig('imgs/neq.svg', bbox_inches='tight')

In [ ]:
# plt.plot(ffff[1700:1950])
plt.plot(ffff[1850:1930])

# so deadzone approx = 25 (pages)
DEADZONE = 19

In [ ]:
is_domain = [True for _ in ffff]
last_freq = [0, 0]
for i, freq in enumerate(ffff):
    two_before = last_freq.pop()
    if two_before == 0:
        is_domain[i] = False
    else:
        ratio = freq / two_before
        if ratio > 1.7:
            # jump up! 
            is_domain[i-1 : i+1] = [False] * 2
            for j in range(i - 2, i - DEADZONE, -1):
                if ffff[j] > freq * .9:
                    break
                is_domain[j] = False
        if ratio < .6:
            # jump down! 
            is_domain[i-1 : i+1] = [False] * 2
            for j in range(i, i + DEADZONE, +1):
                if ffff[j] > two_before * .9:
                    break
                is_domain[j] = False
    last_freq.append(freq)

# domain_p = ppp[is_domain]
# domain_f = ffff[is_domain]

fig, (ax0, ax1) = plt.subplots(2, 1, sharex=True)
x = np.arange(SIZE) / SR * HOP_LEN
ax0.plot(x, eeee * 18, label='amplitude')
ax0.plot(x, ppp * 8, label='pressure')
ax0.plot(x, ffff, label='frequency')
ax0.legend()

ax1.plot(x, ppp * 8, pre_c, label = 'pressure')
ax1.plot(x, ffff, freq_c, label = 'frequency')
last_start = None
span = None
def endRect(end):
    global last_start, span
    if last_start is not None:
        span = ax1.axvspan(x[last_start], x[end], facecolor='r', alpha=0.3)
        last_start = None
for i, is_do in enumerate(is_domain):
    if not is_do:
        if last_start is None:
            last_start = i
    else:
        endRect(i)
endRect(i)
ax1.legend([span], ['removed'])
widePlot(10, 50)
plt.xlabel('time (seconds)')

plt.savefig('imgs/scroll.svg', bbox_inches='tight')

The below cell hand-removes a particularly large dent. 

In [ ]:
plt.plot(range(7600, 7800), ffff[7600:7800])
plt.axvspan(7700, 7752, facecolor='r', alpha=0.5)
for i in range(7700, 7752):
    is_domain[i] = False

In [ ]:
def scatterDomainBend(ax, p, f, start, end, pitch, c, do_bads = True):
    _p = p[start:end]
    f = f[start:end]
    dom = is_domain[start:end]
    _pb = freq2pitch(f) - pitch - .75

    if do_bads:
        p = _p[np.invert(dom)]
        pb = _pb[np.invert(dom)]
        pp = []
        pbpb = []
        for x, y in zip(p, pb):
            if x > 20:
                pp.append(x)
                pbpb.append(y)
        ax.scatter(pp, pbpb, c='k', s=.5, marker='.')

    p = _p[dom]
    pb = _pb[dom]
    pp = []
    pbpb = []
    for x, y in zip(p, pb):
        if x > 20:
            pp.append(x)
            pbpb.append(y)
    sct = ax.scatter(pp, pbpb, c=c, s=.5, marker='.')
    
    ax.grid(which='major')
    ax = plt.gca()
    ax.set_ylim([-3,1])
    return sct

fig, axes = plt.subplots(2, 4, sharey = True, sharex = True)
fig.delaxes(axes[0][-1])
for ax, args in zip([*axes[0][:-1], *axes[1]], SCATTER):
    sct = scatterDomainBend(ax, ppp, ffff, *args[:4])
    lgnd = ax.legend([sct], [args[5]], loc='lower right')
    for handle in lgnd.legendHandles:
        handle.set_sizes([50])
ax = fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.grid(False)
plt.xlabel('pressure (Pa)')
plt.ylabel('pitchbend (semitones)')
widePlot(6, 10)

plt.savefig('imgs/clean_result.svg', bbox_inches='tight')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
fig.subplots_adjust(hspace=0.05)  # adjust space between axes
for args in SCATTER:
#     if args[3] == 'red':
        scatter = scatterDomainBend(ax1, ppp, ffff, *args[:4], False)
        scatter = scatterDomainBend(ax2, ppp, ffff, *args[:4], False)
        scatter.set_label(args[5])
ax1.set_ylim(8.5, 13)
ax2.set_ylim(-3, 1.5)

ax1.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop=False)
ax2.xaxis.tick_bottom()

d = .5
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)

widePlot(7, 10)
plt.xlabel('pressure (Pa)')
plt.ylabel('pitch bend (semitones)', position = (0, 1))
lgnd = ax2.legend()
for handle in lgnd.legendHandles:
    handle.set_sizes([50])
# axes = plt.gca()
# axes.set_xlim([0,100])
# axes.set_ylim([-3,1.5])

plt.savefig('imgs/rainbow_scatter.svg', bbox_inches='tight')

## regressions, parameter finding

In [ ]:
def scatterBendFreq(p, f, start, end, pitch, c, octave_high = False):
    if octave_high: 
        pitch += 12
    p = p[start:end]
    f = f[start:end]
    dom = is_domain[start:end]
    p = p[dom]
    f = f[dom]
    fb = (f - pitch2freq(pitch + .75))
    fq = (f / pitch2freq(pitch + .75))
    pb = freq2pitch(f) - (pitch + .75)
    pp = []
    fbfb = []
    pbpb = []
    fqfq = []
    for x, y, z, u in zip(p, fb, pb, fq):
        if octave_high:
            if x < 20 or y < -500:
                continue
        else:
            if x < 20 or abs(y) > 250 or x > 250:
                continue
        pp.append(np.log(x))
        fbfb.append(y)
#         pbpb.append(z)
        pbpb.append(np.exp(z))
        fqfq.append(u ** 10)
#     plt.scatter(pp, fbfb, c=c, s=1, marker='.')
#     plt.scatter(pp, pbpb, c=c, s=1, marker='.')
    plt.scatter(pp, fqfq, c=c, s=1, marker='.')
#     plt.grid(which='major')
    return pp, fqfq

scatterBendFreq_results = []
for i, args in enumerate(SCATTER):
#     if i >= 3:
        scatterBendFreq_results.append([args[2], 
            scatterBendFreq(ppp, ffff, *args[:4])
        ])
widePlot(5, 8)
# axes = plt.gca()
# axes.set_xlim([0,3])
# axes.set_xlim([0,250])
# axes.set_ylim([-200,50])

In [ ]:
scatterBendFreqHighOctave_results = []
for i, args in enumerate(SCATTER):
#     if i >= 3:
        scatterBendFreqHighOctave_results.append([args[2] + 12, 
            scatterBendFreq(ppp, ffff, *args[:4], True)
        ])
widePlot(5, 8)
# axes = plt.gca()
# axes.set_xlim([0,3])
# axes.set_xlim([0,7.5])
# axes.set_ylim([0,1.2])

In [ ]:
regress_data = scatterBendFreq_results + scatterBendFreqHighOctave_results
assert len(regress_data) == 14   # in case the above scattering code was conditioned

In [ ]:
reg_results = []

# legacy
# for pitch, (X, Y) in regress_data:
#     reg_results.append([pitch, regression(X, Y)])

for i, (pitch, (X, Y)) in enumerate(regress_data):
#     if i in [0, 1, 2, 3, 4, 5, 6]:
#         mix = mixture.GaussianMixture(n_components=2, warm_start=True).fit([*zip(X, Y)])
#         label = mix.predict([*zip(X, Y)])
#         if mix.means_[0][0] < mix.means_[1][0]:
#             choose_label = 0
#         else:
#             choose_label = 1
#         XX = [t for i, t in enumerate(X) if label[i] == choose_label]
#         YY = [t for i, t in enumerate(Y) if label[i] == choose_label]
#     else:
#         XX = X
#         YY = Y
    XX = X
    YY = Y
    
    lm = linear_model.LinearRegression()
#     lm.fit_intercept = False
    model = lm.fit([[t] for t in XX], [[t] for t in YY])
    reg_results.append([pitch, model.coef_[0][0], model.intercept_[0]])
reg_results

In [ ]:
fig, axes = plt.subplots(1, 3, sharey=True)
for r, ax in zip([(0, 14), (0, 7), (7, 14)], axes):
    ax.axhline(1, linewidth = .5, c='k')
    for i in range(*r):
        X, Y = regress_data[i][1]
        c, _, symbol = SCATTER[i % 7][3:]

        YY = [reg_results[i][1] * t + reg_results[i][2] for t in X]
        ax.plot(X, YY, LIGHT[c], linewidth = .5, label=symbol)

        ax.scatter(X, Y, s=.5, c=c)
axes[0].set_title('Both octaves')
axes[1].set_title('Lower octave')
# axes[0].set_xlim([-.5e6, 0.2e8])
# axes[0].set_ylim([-.05, 3])
axes[2].set_title('Higher octave')
# axes[1].set_xlim([-1.0e7, 4.0e8])
# axes[1].set_ylim([-.1, 6])
fig.subplots_adjust(wspace=.3)

handles, labels = ax.get_legend_handles_labels()
ax = fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.grid(False)
lgnd = fig.legend(handles, labels, loc=(.31,.09), prop={'size': 12})
for handle in lgnd.legendHandles:
    handle.set_linewidth(1)
plt.xlabel('$ln($pressure$)$', labelpad=5)
plt.ylabel('frequency quotient \\^{} $10$', labelpad=15)

widePlot(4, 10)

plt.savefig('imgs/bend_regress.svg', bbox_inches='tight')

In [ ]:
mean_slope = np.mean([t[1] for t in reg_results])
def fitIntercept():
    results = []
    for i in range(0, 14):
        X, Y = regress_data[i][1]
        results.append(np.mean(X) - (np.mean(Y) - 1) / mean_slope)
    return results
X = np.array([t[0] for t in reg_results])
intercepts = fitIntercept()
plt.scatter(X, intercepts)

lm = linear_model.LinearRegression()
model = lm.fit([[t[0]] for t in reg_results], [[t] for t in intercepts])
pb_coef = model.coef_[0][0]
pb_inter = model.intercept_[0]
print(pb_coef, pb_inter)

predicted_x_intercept = pb_inter + pb_coef * X

plt.plot(X, predicted_x_intercept)

plt.xlabel('pitch (MIDI)')
plt.ylabel('$ln($pressure$)$ intercept')
plt.xticks([
    *np.array([60, 62, 64, 65, 67, 69, 71]) + 12, 
    *np.array([60, 62, 64, 65, 67, 69, 71]) + 24, 
])
widePlot(3, 10)

plt.savefig('imgs/interc_regress.svg', bbox_inches='tight')

## next step: reverse back to rainbow and overlay

In [ ]:
X = np.array(range(10, 350))
log_X = np.log(X)
ONE_PITCH = freq2pitch(1)

for i, args in enumerate(SCATTER):
    pitch = args[2]
    c = args[3]
    sym = args[5]
    xi_l = predicted_x_intercept[i]
    xi_h = predicted_x_intercept[i + 7]
    
    fq_l = ((log_X - xi_l) * mean_slope + 1) ** .1
    fq_h = ((log_X - xi_h) * mean_slope + 1) ** .1
    pb_l = [freq2pitch(t) - ONE_PITCH for t in fq_l]
    pb_h = [freq2pitch(t) - ONE_PITCH for t in fq_h]
    plt.plot(X, pb_l, c, linewidth = .5)
    plt.plot(X, pb_h, c, linewidth = .5, label=sym)
    
    scatterDomainBend(plt, ppp, ffff, *args[:2], pitch, c, False)
    scatterDomainBend(plt, ppp, ffff, *args[:2], pitch+12, c, False)
widePlot(10, 9)
axes = plt.gca()
axes.set_xlim([10,320])
axes.set_ylim([-3,.8])
plt.xlabel('pressure (Pa)')
plt.ylabel('pitch bend (semitones)')
plt.legend()

plt.savefig('imgs/rainbow_overlay.svg', bbox_inches='tight')

## !!! only three free parameters! 
and one of them is "10"

## Failure: study amplitude

In [ ]:
# Legacy code

# plt.plot(ffff[230:1300])
# # plt.plot(ffff[1550:2200])
# # plt.plot(ffff[2200:2800])
# # plt.plot(ffff[2800:3450])
# widePlot()

# plt.plot(ffff[230:580])
# plt.plot(ffff[580:960])

# # scatterBend(ppp, ffff, 230, 580, 83, 'r')

# scatterBend(ppp, ffff, 580, 1300, 83, 'r')
# scatterBend(ppp, ffff, 1550, 2200, 82, 'g')
# scatterBend(ppp, ffff, 2200, 2800, 80, 'b')
# scatterBend(ppp, ffff, 2800, 3450, 78, 'k')
# plt.grid(which='major')

In [ ]:
def scatterVelo(p, e, start, end, _, c):
    p = p[start:end]
    e = e[start:end]
    pp = []
    ee = []
    for x, y in zip(p, e):
        if x > 20:
#             if x < 100:
                pp.append(x ** 1)
                ee.append(y ** 1)
    plt.scatter(pp, ee, c=c, s=.5, marker='.')

for i, args in enumerate(SCATTER):
    scatterVelo(ppp, eeee, *args[:4])
#     if i == 6:
#         scatterVelo(ppp, eeee, *args[:3], 'k')
# widePlot(10, 10)
widePlot()

Total failure. 

## octave threshold

In [ ]:
# Hand labeling from note-wise pressure-pitch scatter. 
OCTV_THRES = [
    [60, [62,   64,   66,   66, ],  [51, 55, 57, 54]],
    [62, [80,   101, 84,   79, ],  [83, 80, 82, 80]],
    [64, [104, 97,   112, 101,], [75, 73, 74, 73]],
    [65, [122, 99,   91,   95, ],  [79, 72, 79, 79]],
    [67, [159, 141, 122, 126,], [149, 106, 99, 96]],
    [69, [236,      216, 225,], [212,  186, 188]],
    [71, [], [201]],
]


In [ ]:
def scatterOctave(s):
    x_b = []
    y_b = []
    x_r = []
    y_r = []
    x = []
    y = []
    c = []
    for pitch, ups, downs in OCTV_THRES:
        for things, color in zip([ups, downs], ['b', 'r']):
            for thing in things:
                c.append(color)
                x.append(pitch)
                y.append(np.log(thing))
                if color == 'b':
                    x_b.append(pitch)
                    y_b.append(np.log(thing))
                else:
                    x_r.append(pitch)
                    y_r.append(np.log(thing))
    plt.scatter(x_b, y_b, s=s, marker='o', facecolors='none', edgecolors='b', label = 'upward')
    plt.scatter(x_r, y_r, s=s, marker='o', facecolors='none', edgecolors='r', label = 'downward')
    return x, y, c

# single line fit - ignores hysteresis

x, y, c = scatterOctave(s=20)

lm = linear_model.LinearRegression()
model = lm.fit([[t] for t in x], [[t] for t in y])
ot_coef = model.coef_[0][0]
ot_inter = model.intercept_[0]
print(ot_coef, ot_inter)
y_hat = ot_inter + ot_coef * np.array(x)
plt.plot(x, y_hat, c='k')
plt.xlabel('pitch (MIDI)')
plt.ylabel('$ln($pressure$)$')
plt.legend()


In [ ]:
x, y, c = scatterOctave(s=20)

lm = linear_model.LinearRegression()
model = lm.fit([[t, 0 if tt == 'r' else 1] for t, tt in zip(x, c)], [[t] for t in y])
ot_coef = model.coef_[0][0]
ot_c_coef = model.coef_[0][1]
ot_inter = model.intercept_[0]
print(ot_coef, ot_c_coef, ot_inter)
y_hat = ot_inter + ot_coef * np.array(x)
plt.plot(x, y_hat, 'r')
plt.plot(x, y_hat + ot_c_coef, 'b')

plt.xlabel('pitch (MIDI)')
plt.ylabel('$ln($pressure$)$')
plt.xticks([60, 62, 64, 65, 67, 69, 71])
plt.legend()

plt.savefig('imgs/octave_thresholds.svg', bbox_inches='tight')